<a href="https://colab.research.google.com/github/farnoushTalebi/HW-01-FarnooshTalebi-4003614024/blob/main/BD_Cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python3-kafka

In [ ]:
!pip install kafka-python

In [2]:
!pip install kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 207 kB 13.6 MB/s 


In [ ]:
from kafka import KafkaConsumer
consumer = KafkaConsumer('test_topic')
for msg in consumer:
     print (msg)

In [ ]:
!pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 5.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np 

# Reading the data
df = pd.read_csv("test.csv")
print(df.shape)
print(df.info())
df.head(5)

(3992, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3992 entries, 0 to 3991
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date/Time  3992 non-null   object 
 1   Lat        3992 non-null   float64
 2   Lon        3992 non-null   float64
 3   Base       3992 non-null   object 
dtypes: float64(2), object(2)
memory usage: 124.9+ KB
None


,Date/Time,Lat,Lon,Base
0,9/8/2014 0:01,40.2201,-74.0021,B02512
1,9/16/2014 0:01,40.7500,-74.0027,B02512
2,9/1/2014 0:03,40.7559,-73.9864,B02512
3,9/29/2014 0:06,40.7450,-73.9889,B02512
4,7/1/2014 0:11,40.8145,-73.9444,B02512


In [ ]:
import datetime

df['UUID'] = df.index
df['Date'] = pd.to_datetime(df['Date/Time'])
df['Date'] = df['Date'].dt.strftime('%m.%d.%Y %H')
df['weekofyear'] = pd.DatetimeIndex(df['Date']).weekofyear
df['hour'] = pd.DatetimeIndex(df['Date']).hour
df['12hours'] = df['hour']%2
df['value'] = df['Date/Time'].astype(str) + "|" + df['Lat'].astype(str) + df['Lon'].astype(str) + "|" + df['Base'].astype(str)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


,Date/Time,Lat,Lon,Base,UUID,Date,weekofyear,hour,12hours,value
0,9/8/2014 0:01,40.2201,-74.0021,B02512,0,09.08.2014 00,37,0,0,9/8/2014 0:01|40.2201-74.0021|B02512
1,9/16/2014 0:01,40.7500,-74.0027,B02512,1,09.16.2014 00,38,0,0,9/16/2014 0:01|40.75-74.0027|B02512
2,9/1/2014 0:03,40.7559,-73.9864,B02512,2,09.01.2014 00,36,0,0,9/1/2014 0:03|40.7559-73.9864|B02512
3,9/29/2014 0:06,40.7450,-73.9889,B02512,3,09.29.2014 00,40,0,0,9/29/2014 0:06|40.745-73.9889|B02512
4,7/1/2014 0:11,40.8145,-73.9444,B02512,4,07.01.2014 00,27,0,0,7/1/2014 0:11|40.8145-73.9444|B02512
...,...,...,...,...,...,...,...,...,...,...
3987,9/4/2014 17:15,40.7647,-73.9957,B02512,3987,09.04.2014 17,36,17,1,9/4/2014 17:15|40.7647-73.9957|B02512
3988,9/4/2014 17:16,40.7196,-73.9981,B02512,3988,09.04.2014 17,36,17,1,9/4/2014 17:16|40.7196-73.9981|B02512
3989,9/4/2014 17:16,40.7417,-73.9856,B02512,3989,09.04.2014 17,36,17,1,9/4/2014 17:16|40.7417-73.9856|B02512
3990,9/4/2014 17:16,40.7507,-73.9700,B02512,3990,09.04.2014 17,36,17,1,9/4/2014 17:16|40.7507-73.97|B02512


In [ ]:
from cassandra.cluster import Cluster

def create_connection():
    cluster = Cluster(['127.0.0.1'])
    return cluster.connect('mykeyspace')

def create_keyspace():
    session.execute("CREATE KEYSPACE 'mykeyspace' WITH replication = {'class': 'Strategy name', 'replication_factor' : 'No.Of replicas'} ")

def create_table_week(session, UUID, date, lat, lon, Base, week):
  session.execute("CREATE TABLE weektable("
  + "UUID varint, date text, lat float, lon float, Base text, week varint, "
  + "PRIMARY KEY ((UUID, date), week)"
  + "WITH CLUSTERING ORDER BY (week desc);")

def create_table_latlon(session, UUID, date, lat, lon, Base):
  session.execute("CREATE TABLE latlon("
  + "UUID varint, date text, lat float, lon float, Base text, "
  + "PRIMARY KEY (lat, lon);")

def create_table_value(session, UUID, value):
  session.execute("CREATE TABLE value("
  + "UUID varint, value text, "
  + "PRIMARY KEY (UUID);")

def create_table_12hour(session, UUID, date, lat, lon, Base, _12hour):
  session.execute("CREATE TABLE _12hour("
  + "UUID varint, date text, lat float, lon float, Base text, _12hour varint, "
  + "PRIMARY KEY ((UUID, date), _12hour);")

def get_from_latlon(session, lat, lon):
  result = session.execute("SELECT * FROM latlon WHERE lat = %f, lon = %f", [lat, lon])
  for i in result:
      print (i.UUID, i.Date, i.Base)

def get_from_week(session, lat, lon):
  result = session.execute("SELECT * FROM weektable DISTINCT week WHERE lat = %f, lon = %f", [lat, lon])
  for i in result:
      print (i.UUID, i.Date)
  
def get_last_week(session):
  result = session.execute("SELECT * FROM weektable DISTINCT week LIMIT 1;")
  return result
  #for i in result:
   #   print (i.UUID, i.Date)

def get_5_last_week(session):
  result = session.execute("SELECT * FROM weektable DISTINCT week LIMIT 5;")
  return result

def main():
    session = create_connection()
    UUID = df['UUID']
    _12hour = df['12hour']
    lat = df['Lat']
    lon = df['Lon']
    Base = df['Base']
    week = df['weekofyear']
    date = df['Date']
    value = df['value']

    create_table_week(session, UUID, date, lat, lon, Base, week)
    create_table_latlon(session, UUID, date, lat, lon, Base)
    create_table_value(session, UUID, value)
    create_table_12hour(session, UUID, date, lat, lon, Base, _12hour)

    get_from_latlon(session, 40.7674, -73.9957)
    get_from_week(session, 40.7674, -73.9957)

    train = get_5_last_week(session)
    test = get_last_week(session)

if __name__ == "__main__":
    main()

ModuleNotFoundError: ignored

In [ ]:
target_column_train = ['Base'] 
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

target_column_test = ['Base'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

KeyError: ignored

In [ ]:
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
dtree = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)
dtree.fit(X_train, y_train)
pred_train_tree= dtree.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_tree)))
print(r2_score(y_train, pred_train_tree))

pred_test_tree= dtree.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_tree))) 
print(r2_score(y_test, pred_test_tree))

In [ ]:
dtree1 = DecisionTreeRegressor(max_depth=2)
dtree2 = DecisionTreeRegressor(max_depth=5)
dtree1.fit(X_train, y_train)
dtree2.fit(X_train, y_train)

tr1 = dtree1.predict(X_train)
tr2 = dtree2.predict(X_train) 

y1 = dtree1.predict(X_test)
y2 = dtree2.predict(X_test)

In [ ]:
print(np.sqrt(mean_squared_error(y_train,tr1))) 
print(r2_score(y_train, tr1))

print(np.sqrt(mean_squared_error(y_test,y1))) 
print(r2_score(y_test, y1))

In [ ]:

print(np.sqrt(mean_squared_error(y_train,tr2))) 
print(r2_score(y_train, tr2))

print(np.sqrt(mean_squared_error(y_test,y2))) 
print(r2_score(y_test, y2))

In [ ]:
#RF model
model_rf = RandomForestRegressor(n_estimators=5000, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

pred_test_rf = model_rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))